**Setting up the network**

In [957]:
import pandas as pd

import os

import pypsa

In [958]:

# Specify the path to the directory you want to set as the working directory
new_directory = '/Users/danialriaz/PyPSA-Eur'

# Change the current working directory to the new directory
os.chdir(new_directory)

n = pypsa.Network("results/test-elec/networks/elec_s_8_ec_lcopt_Co2L-1H.nc")


INFO:pypsa.io:Imported network elec_s_8_ec_lcopt_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


**Load at each hour (MWh)**

In [959]:
# Transform n.buses_t.marginal_price into a DataFrame
loads_df = pd.DataFrame(n.loads_t.p_set)

loads_df.reset_index(inplace=True)

# Rename the 'index' column to 'timestamp'
loads_df.rename(columns={'snapshot': 'timestamp'}, inplace=True)

# Melt the DataFrame
loads_long = pd.melt(loads_df, id_vars=['timestamp'], var_name='country_bus', value_name='load')

# Split the values in the 'technology' column
loads_long[['country', 'bus']] = loads_long['country_bus'].str.split(' ', expand=True)

# Drop the original 'technology' column
loads_long.drop(columns=['country_bus'], inplace=True)

# Edit country names
loads_long['country'] = loads_long['country'].str[:-1]

loads_long.head(5)


,timestamp,load,country,bus
0,2013-01-01 00:00:00,14959.498867,DE,0
1,2013-01-01 01:00:00,14355.448782,DE,0
2,2013-01-01 02:00:00,13781.622456,DE,0
3,2013-01-01 03:00:00,13569.204534,DE,0
4,2013-01-01 04:00:00,13334.650900,DE,0


In [960]:
# n.loads_t.p.filter(like='DE').sum().sum()
 
loads_grouped = loads_long.groupby(['country'])['load'].sum().reset_index()
loads_grouped['load'] *= (1/1000)  # Convert MWh to GWh
loads_grouped.rename(columns={'load': 'load (GWh)'}, inplace=True)  # Rename the column to indicate GWh
loads_grouped['load (GWh)'] = loads_grouped['load (GWh)']
loads_grouped.round(1)


,country,load (GWh)
0,DE,509158.9
1,FR,492192.9
2,NO,128144.0
3,SE,139580.3


**Price at each hour (€/MWh)**

Mean price per country
Graph per country (excluding the peak price)
Explore: what causes the price hike in Jan?

In [961]:
prices_df = pd.DataFrame(n.buses_t.marginal_price)
prices_df.reset_index(inplace=True)
prices_df.rename(columns={'snapshot': 'timestamp'}, inplace=True)
# Filter out columns containing 'H2' in their names
prices_df = prices_df[[col for col in prices_df.columns if 'H2' not in col]]
prices_long = pd.melt(prices_df, id_vars=['timestamp'], var_name='country_bus', value_name='marginal_price')
prices_long[['country', 'bus']] = prices_long['country_bus'].str.split(' ', expand=True)
prices_long.drop(columns=['country_bus'], inplace=True)
prices_long['country'] = prices_long['country'].str[:-1]

prices_grouped = prices_long.groupby(['country', 'timestamp'])['marginal_price'].mean().reset_index()

prices_long = prices_long[['timestamp','country', 'bus', 'marginal_price']]
prices_long = prices_long.sort_values(by= ['timestamp', 'country', 'bus'])

prices_long.head(5)



,timestamp,country,bus,marginal_price
0,2013-01-01,DE,0,70.430919
8760,2013-01-01,DE,1,70.430925
17520,2013-01-01,DE,2,70.430924
26280,2013-01-01,FR,0,70.431060
35040,2013-01-01,FR,1,70.431065


**Createing Weighted prices**

In [962]:
bus_level_load = pd.merge(prices_long, loads_long, on=['country','bus','timestamp'], how='outer')

bus_level_load = bus_level_load.sort_values(by= ['timestamp', 'country', 'bus'])

# Calculate the load-weighted marginal price
bus_level_load['load_sum'] = bus_level_load.groupby(['timestamp', 'country'])['load'].transform('sum')
bus_level_load['load_weights'] = (bus_level_load['load'] / bus_level_load['load_sum'])
bus_level_load['weighted_price'] = (bus_level_load['load_weights'] * bus_level_load['marginal_price'])
bus_level_load['sum_weighted_price'] = bus_level_load.groupby(['timestamp', 'country'])['weighted_price'].transform('sum')

weighted_price = bus_level_load.groupby(['country', 'timestamp'])['weighted_price'].sum().reset_index()
bus_level_load.head(5).round(1)


,timestamp,country,bus,marginal_price,load,load_sum,load_weights,weighted_price,sum_weighted_price
0,2013-01-01,DE,0,70.4,14959.5,38617.6,0.4,27.3,70.4
8760,2013-01-01,DE,1,70.4,10129.4,38617.6,0.3,18.5,70.4
17520,2013-01-01,DE,2,70.4,13528.7,38617.6,0.4,24.7,70.4
26280,2013-01-01,FR,0,70.4,22542.4,57243.0,0.4,27.7,70.4
35040,2013-01-01,FR,1,70.4,15048.0,57243.0,0.3,18.5,70.4


In [963]:
# weighted_price.to_excel('weighted_price.xlsx', index=False)

**Generation at each hour of dispatch and storage units (MWh)**

In [964]:
## GENERATORS ##

# Convert n.generators_t.p into a DataFrame
generators_df = n.generators_t.p.reset_index()

# Rename the 'index' column to 'timestamp'
generators_df.rename(columns={'snapshot': 'timestamp'}, inplace=True)

# Melt the DataFrame
generators_df_long = pd.melt(generators_df, id_vars=['timestamp'], var_name='bus_tech', value_name='generation')

# # Split the values in the 'technology' column
generators_df_long[['country', 'bus', 'technology']] = generators_df_long['bus_tech'].str.split(' ', expand=True)

# # Drop the original 'technology' column
generators_df_long.drop(columns=['bus_tech'], inplace=True)

# # Assuming df is your DataFrame
generators_df_long['country'] = generators_df_long['country'].str[:-1]

# Replace 'technology' values containing 'wind' with 'wind'
generators_df_long.loc[generators_df_long['technology'].str.contains('wind'), 'technology'] = 'wind'


## STORAGE ##

# Convert n.storage_units_t.p into a DataFrame
hydro_gen_df = n.storage_units_t.p_dispatch.reset_index()

# Rename the 'index' column to 'timestamp'
hydro_gen_df.rename(columns={'snapshot': 'timestamp'}, inplace=True)

# Melt the DataFrame
hydro_gen_df_long = pd.melt(hydro_gen_df, id_vars=['timestamp'], var_name='bus_tech', value_name='generation')

# # Split the values in the 'technology' column
hydro_gen_df_long[['country', 'bus', 'technology']] = hydro_gen_df_long['bus_tech'].str.split(' ', expand=True)

# # Drop the original 'technology' column
hydro_gen_df_long.drop(columns=['bus_tech'], inplace=True)

# # Assuming df is your DataFrame
hydro_gen_df_long['country'] = hydro_gen_df_long['country'].str[:-1]

hydro_gen_df_long['technology'] = hydro_gen_df_long['technology'].replace({'hydro': 'reservoir'})


## MERGE ##

generation_df = pd.merge(generators_df_long, hydro_gen_df_long, on=['country','timestamp', 'bus','technology', 'generation'], how='outer')

generation_df = generation_df[['timestamp', 'country', 'bus', 'technology', 'generation']]

generation_df.head(10).round(1)

,timestamp,country,bus,technology,generation
0,2013-01-01,DE,0,CCGT,0.6
1,2013-01-01,DE,0,OCGT,0.0
2,2013-01-01,DE,0,PHS,0.0
3,2013-01-01,DE,0,coal,0.0
4,2013-01-01,DE,0,lignite,0.0
5,2013-01-01,DE,0,nuclear,22884.0
6,2013-01-01,DE,0,reservoir,252.5
7,2013-01-01,DE,0,ror,1736.4
8,2013-01-01,DE,0,solar,0.0
9,2013-01-01,DE,1,CCGT,0.2


In [965]:
# weighted_price.to_excel('weighted_price.xlsx', index=False)

In [966]:
# Extract the hour from the 'timestamp' column and create a new 'hour' column
generation_df['hour'] = pd.to_datetime(generation_df['timestamp']).dt.hour

avg_gen = generation_df.groupby(['country', 'hour', 'technology'])['generation'].mean().reset_index()

avg_gen.head(5)

,country,hour,technology,generation
0,DE,0,CCGT,494.200504
1,DE,0,OCGT,0.000459
2,DE,0,PHS,13.377756
3,DE,0,coal,0.000996
4,DE,0,lignite,0.001558


In [967]:
# avg_gen.to_excel('average_generation.xlsx', index=False)

**Generation by country**

In [968]:
gen_grouped = generation_df.groupby(['country'])['generation'].sum().reset_index()
# gen_grouped['generation'] *= (1/1000)  # Convert MWh to GWh
# gen_grouped.rename(columns={'generation': 'generation (GWh)'}, inplace=True)  # Rename the column to indicate GWh
# gen_grouped['generation (GWh)'] = gen_grouped['generation (GWh)']
gen_grouped.round(1)


,country,generation
0,DE,343075797.8
1,FR,457356011.2
2,NO,389765926.8
3,SE,86737567.0


**Revenue by bus: Merge generation and prices**

In [969]:
revenue_df = pd.merge(generation_df, prices_long, on=['timestamp', 'country','bus'], how='outer')
revenue_df['revenue'] = (revenue_df['generation'] * revenue_df['marginal_price'])
revenue_df = revenue_df[['timestamp', 'country', 'bus', 'technology', 'generation', 'marginal_price', 'revenue']]
revenue_df.head(10).round(1)

,timestamp,country,bus,technology,generation,marginal_price,revenue
0,2013-01-01,DE,0,CCGT,0.6,70.4,45.4
1,2013-01-01,DE,0,OCGT,0.0,70.4,0.0
2,2013-01-01,DE,0,PHS,0.0,70.4,0.1
3,2013-01-01,DE,0,coal,0.0,70.4,0.1
4,2013-01-01,DE,0,lignite,0.0,70.4,0.1
5,2013-01-01,DE,0,nuclear,22884.0,70.4,1611744.5
6,2013-01-01,DE,0,reservoir,252.5,70.4,17783.8
7,2013-01-01,DE,0,ror,1736.4,70.4,122293.7
8,2013-01-01,DE,0,solar,0.0,70.4,0.0
9,2013-01-01,DE,1,CCGT,0.2,70.4,11.1


**Base Price**

In [970]:
base_price = weighted_price.groupby(['country'])['weighted_price'].mean().reset_index()
base_price.rename(columns={'weighted_price': 'base_price'}, inplace=True)
base_price

,country,base_price
0,DE,82.832806
1,FR,84.022096
2,NO,74.097524
3,SE,84.049630


**Market Value and Value Factor by technology**

In [971]:
mv_df = revenue_df.groupby(['country', 'technology'])[['generation', 'revenue']].sum().reset_index()
mv_df['market_value'] = (mv_df['revenue'] / mv_df['generation'])
mv_df = mv_df[mv_df['generation'] != 0]

mv_vf_df = pd.merge(base_price, mv_df, on=['country'], how='outer')
mv_vf_df['value_factor'] = (mv_vf_df['market_value'] / mv_vf_df['base_price'])

## ADD COUNTRY LEVEL generation %s ##

mv_vf_df['country_sum_gen'] = mv_vf_df.groupby('country')['generation'].transform('sum')
mv_vf_df['gen_perc_mix(%)'] = (mv_vf_df['generation']/mv_vf_df['country_sum_gen']) * 100
mv_vf_df.drop(columns=['country_sum_gen'], inplace=True)
mv_vf_df['gen_perc_mix(%)'] = mv_vf_df['gen_perc_mix(%)'].round(1)
mv_vf_df = mv_vf_df.sort_values(by= ['country', 'gen_perc_mix(%)'], ascending=False)

mv_vf_df.head(5)

,country,base_price,technology,generation,revenue,market_value,value_factor,gen_perc_mix(%)
33,SE,84.04963,reservoir,4.892635e+07,4.732072e+09,96.718262,1.150728,56.4
27,SE,84.04963,CCGT,2.627247e+07,2.213666e+09,84.257998,1.002479,30.3
34,SE,84.04963,ror,6.439249e+06,5.393124e+08,83.753919,0.996482,7.4
35,SE,84.04963,solar,4.522556e+06,3.292064e+08,72.792128,0.866061,5.2
28,SE,84.04963,OCGT,5.047266e+05,3.663631e+08,725.864547,8.636142,0.6


**Variable Costs and AFC of all technologies (inlc. PHS) (€/MWh)**

In [972]:
## VC ##

# Convert n.generators_t.p into a DataFrame
mc_data = n.generators.marginal_cost.reset_index()

# # Split the values in the 'technology' column
mc_data[['country', 'bus', 'technology']] = mc_data['Generator'].str.split(' ', expand=True)

# # Drop the original 'technology' column
mc_data.drop(columns=['Generator'], inplace=True)

# # Assuming df is your DataFrame
mc_data['country'] = mc_data['country'].str[:-1]

# Replace 'technology' values containing 'wind' with 'wind'
mc_data.loc[mc_data['technology'].str.contains('wind'), 'technology'] = 'wind'

# group bus level data
mc_data_tech = mc_data.groupby(['country', 'technology'])['marginal_cost'].mean().reset_index()
mc_data_tech = mc_data_tech.sort_values(by= ['country', 'marginal_cost','technology'], ascending=False)

# Convert n.generators_t.p into a DataFrame
mc_data_st = n.storage_units.marginal_cost.reset_index()

# # Split the values in the 'technology' column
mc_data_st[['country', 'bus', 'technology']] = mc_data_st['StorageUnit'].str.split(' ', expand=True)

# # Drop the original 'technology' column
mc_data_st.drop(columns=['StorageUnit'], inplace=True)

# # Assuming df is your DataFrame
mc_data_st['country'] = mc_data_st['country'].str[:-1]

# Replace 'technology' values containing 'wind' with 'wind'
mc_data_st.loc[mc_data_st['technology'].str.contains('wind'), 'technology'] = 'wind'

# group bus level data
mc_data_tech_st = mc_data_st.groupby(['country', 'technology'])['marginal_cost'].mean().reset_index()
mc_data_tech_st = mc_data_tech_st.sort_values(by= ['country', 'marginal_cost','technology'], ascending=False)

marg_cost_df = pd.merge(mc_data_tech, mc_data_tech_st, on=['country','technology', 'marginal_cost'], how='outer')
marg_cost_df = marg_cost_df.sort_values(by= ['country', 'marginal_cost'], ascending=False)

marg_cost_df.rename(columns={'marginal_cost': 'marginal_cost (€/MWh)'}, inplace=True)

## AFC ##

# Convert n.generators_t.p into a DataFrame
afc_data = n.generators.capital_cost.reset_index()

# # Split the values in the 'technology' column
afc_data[['country', 'bus', 'technology']] = afc_data['Generator'].str.split(' ', expand=True)

# # Drop the original 'technology' column
afc_data.drop(columns=['Generator'], inplace=True)

# # Assuming df is your DataFrame
afc_data['country'] = afc_data['country'].str[:-1]

# Replace 'technology' values containing 'wind' with 'wind'
afc_data.loc[mc_data['technology'].str.contains('wind'), 'technology'] = 'wind'

# group bus level data
afc_data_tech = afc_data.groupby(['country', 'technology'])['capital_cost'].mean().reset_index()
afc_data_tech = afc_data_tech.sort_values(by= ['country', 'capital_cost','technology'], ascending=False)
afc_data_tech['capital_cost'] = afc_data_tech['capital_cost']


# Convert n.generators_t.p into a DataFrame
afc_data_st = n.storage_units.capital_cost.reset_index()

# # Split the values in the 'technology' column
afc_data_st[['country', 'bus', 'technology']] = afc_data_st['StorageUnit'].str.split(' ', expand=True)

# # Drop the original 'technology' column
afc_data_st.drop(columns=['StorageUnit'], inplace=True)

# # Assuming df is your DataFrame
afc_data_st['country'] = afc_data_st['country'].str[:-1]

# group bus level data
afc_data_tech_st = afc_data_st.groupby(['country', 'technology'])['capital_cost'].mean().reset_index()
afc_data_tech_st = afc_data_tech_st.sort_values(by= ['country', 'capital_cost','technology'], ascending=False)
afc_data_tech_st['capital_cost'] = afc_data_tech_st['capital_cost']

afc_df = pd.merge(afc_data_tech, afc_data_tech_st, on=['country','technology', 'capital_cost'], how='outer')
afc_df = afc_df.sort_values(by= ['country', 'capital_cost'], ascending=False)

afc_df.rename(columns={'capital_cost': 'AFC (€/MW)'}, inplace=True)

costs_df = pd.merge(marg_cost_df, afc_df, on=['country','technology'], how='outer')

costs_df['technology'] = costs_df['technology'].replace({'hydro': 'reservoir'})

costs_df.head(5).round(2)

,country,technology,marginal_cost (€/MWh),AFC (€/MW)
0,DE,CCGT,70.43,119304.52
1,DE,OCGT,144.32,62163.31
2,DE,PHS,0.01,182698.73
3,DE,coal,106.46,132163.71
4,DE,reservoir,0.01,182698.73


**Optimal Capacity (GW)**

In [973]:
opt_df = n.generators.p_nom_opt.reset_index()

opt_df[['country', 'bus', 'technology']] = opt_df['Generator'].str.split(' ', expand=True)

opt_df.drop(columns=['Generator'], inplace=True)

opt_df['country'] = opt_df['country'].str[:-1]

opt_df.rename(columns={'p_nom_opt': 'capacity'}, inplace=True)

opt_df.loc[opt_df['technology'].str.contains('wind'), 'technology'] = 'wind'

opt_df_tech = opt_df.groupby(['country','technology'])['capacity'].sum().reset_index()

opt_df_tech['capacity'] = opt_df_tech['capacity']

storage_cap_df = n.storage_units.p_nom.reset_index()
storage_cap_df[['country', 'bus', 'technology']] = storage_cap_df['StorageUnit'].str.split(' ', expand=True)
storage_cap_df.drop(columns=['StorageUnit'], inplace=True)
storage_cap_df['country'] = storage_cap_df['country'].str[:-1]
storage_cap_df.rename(columns={'p_nom':'capacity'}, inplace=True)
storage_cap_df = storage_cap_df.groupby(['country','technology'])['capacity'].sum().reset_index()


cap_df = pd.merge(storage_cap_df, opt_df_tech, on=['country', 'technology', 'capacity'], how='outer')

cap_df['technology'] = cap_df['technology'].replace({'hydro': 'reservoir'})
cap_df['capacity'] /= 1000

cap_df.rename(columns={'capacity': 'capacity (GW)'}, inplace=True)

## Add country wise capacity percentages

cap_df['Country_Sum_Capacity'] = cap_df.groupby('country')['capacity (GW)'].transform('sum')
cap_df['capacity_perc_mix(%)'] = (cap_df['capacity (GW)'] / cap_df['Country_Sum_Capacity']) * 100
cap_df.drop(columns=['Country_Sum_Capacity'], inplace=True)
cap_df['capacity_perc_mix(%)'] = cap_df['capacity_perc_mix(%)'].round(1)
cap_df = cap_df.sort_values(by=['country', 'capacity_perc_mix(%)'], ascending=False)

cap_df.head(5)

,country,technology,capacity (GW),capacity_perc_mix(%)
31,SE,reservoir,10.784187,44.3
28,SE,OCGT,4.677707,19.2
35,SE,solar,4.459432,18.3
27,SE,CCGT,3.045027,12.5
34,SE,ror,1.279000,5.3


**Calculating Profits**

In [974]:
esm_df = pd.merge(mv_vf_df, costs_df, on=['country','technology'], how='outer')
esm_df = pd.merge(esm_df, cap_df, on=['country','technology'], how='outer')

esm_df['ST profit (€ mil)'] = ((esm_df['revenue'] - (esm_df['marginal_cost (€/MWh)']*esm_df['generation']))/ 1000000)
esm_df['total FOC (€ mil)'] = (((esm_df['AFC (€/MW)']* esm_df['capacity (GW)']*1000))/1000000)
esm_df['LT profit (€ mil)'] = (esm_df['ST profit (€ mil)'] - esm_df['total FOC (€ mil)'])

esm_df = esm_df[esm_df['capacity (GW)'] != 0]

esm_df.head(5)

,country,base_price,technology,generation,revenue,market_value,value_factor,gen_perc_mix(%),marginal_cost (€/MWh),AFC (€/MW),capacity (GW),capacity_perc_mix(%),ST profit (€ mil),total FOC (€ mil),LT profit (€ mil)
0,DE,82.832806,CCGT,1.399272e+07,1.193172e+09,85.270943,1.029434,4.1,70.430548,119304.517221,1.740621,2.3,207.657432,207.664001,-0.006569
1,DE,82.832806,OCGT,3.232979e+06,1.004701e+09,310.766267,3.751729,0.9,144.321980,62163.310332,8.837574,11.8,538.110945,549.372864,-11.261920
2,DE,82.832806,PHS,3.256026e+06,8.365198e+08,256.914334,3.101601,0.9,0.009888,182698.734592,7.599220,10.2,836.487637,1388.367878,-551.880241
3,DE,82.832806,coal,3.121763e+01,3.325219e+03,106.517346,1.285932,0.0,106.460214,132163.708310,0.000010,0.0,0.000002,0.001376,-0.001374
4,DE,82.832806,lignite,4.440745e+01,4.083662e+03,91.958939,1.110175,0.0,87.985226,193840.105522,0.000009,0.0,0.000176,0.001672,-0.001496


Calculating FLH and categorizing


In [975]:

esm_df['FLH'] = (esm_df['generation'] / (esm_df['capacity (GW)']*1000))

import pandas as pd

def categorize_flh(flh):
    if flh > 7000:
        return 'Base load (>7000 FLH)'
    elif 4000 <= flh <= 7000:
        return 'Mid load (4000-7000 FLH)'  # Added a mid category
    elif 1000 <= flh < 4000:
        return 'Peak load (1000-4000 FLH)'  # Adjusted range
    else:  # flh < 1000
        return 'Super peak (<1000 FLH)'

esm_df['FLH category'] = esm_df['FLH'].apply(categorize_flh)


2 calculations for LCOE


In [976]:
esm_df['total VOC (€ mil)'] = ((esm_df['generation'] * esm_df['marginal_cost (€/MWh)'])/1e6)

esm_df['technology'] = esm_df['technology'].replace({'coal': 'hard coal'})

esm_df['LCOE'] = ((esm_df['total FOC (€ mil)']*1e6 + esm_df['total VOC (€ mil)']) / esm_df['generation'])

esm_df['LCOE 2'] = (esm_df['AFC (€/MW)'] / esm_df['FLH'])

esm_df = esm_df.sort_values(by=['country', 'LCOE'], ascending=False)

esm_df.round(1).head(10)

,country,base_price,technology,generation,revenue,market_value,value_factor,gen_perc_mix(%),marginal_cost (€/MWh),AFC (€/MW),capacity (GW),capacity_perc_mix(%),ST profit (€ mil),total FOC (€ mil),LT profit (€ mil),FLH,FLH category,total VOC (€ mil),LCOE,LCOE 2
28,SE,84.0,OCGT,504726.6,3.663631e+08,725.9,8.6,0.6,149.7,62163.3,4.7,19.2,290.8,290.8,-0.0,107.9,Super peak (<1000 FLH),75.6,576.1,576.1
29,SE,84.0,PHS,72040.1,1.574243e+07,218.5,2.6,0.1,0.0,182698.7,0.1,0.4,15.7,16.8,-1.1,783.0,Super peak (<1000 FLH),0.0,233.3,233.3
32,SE,84.0,nuclear,149.8,1.261610e+04,84.2,1.0,0.0,14.0,526254.8,0.0,0.0,0.0,0.0,-0.0,7353.3,Base load (>7000 FLH),0.0,71.6,71.6
34,SE,84.0,ror,6439249.3,5.393124e+08,83.8,1.0,7.4,0.0,308170.4,1.3,5.3,539.2,394.1,145.1,5034.6,Mid load (4000-7000 FLH),0.1,61.2,61.2
30,SE,84.0,hard coal,10.7,1.200400e+03,112.5,1.3,0.0,106.4,132163.7,0.0,0.0,0.0,0.0,-0.0,2696.0,Peak load (1000-4000 FLH),0.0,49.0,49.0
31,SE,84.0,lignite,12.0,1.144600e+03,95.4,1.1,0.0,95.9,193840.1,0.0,0.0,-0.0,0.0,-0.0,4590.7,Mid load (4000-7000 FLH),0.0,42.2,42.2
33,SE,84.0,reservoir,48926350.2,4.732072e+09,96.7,1.2,56.4,0.0,182698.7,10.8,44.3,4731.6,1970.3,2761.3,4536.9,Mid load (4000-7000 FLH),0.5,40.3,40.3
35,SE,84.0,solar,4522555.5,3.292064e+08,72.8,0.9,5.2,0.0,39296.5,4.5,18.3,329.1,175.2,153.9,1014.2,Peak load (1000-4000 FLH),0.1,38.7,38.7
27,SE,84.0,CCGT,26272472.9,2.213666e+09,84.3,1.0,30.3,70.4,119304.5,3.0,12.5,363.3,363.3,-0.0,8628.0,Base load (>7000 FLH),1850.4,13.8,13.8
20,NO,74.1,PHS,64144.3,1.642979e+07,256.1,3.5,0.0,0.0,182698.7,0.3,0.4,16.4,51.6,-35.1,227.2,Super peak (<1000 FLH),0.0,804.1,804.1


In [977]:
esm_df.to_excel('esm-0.xlsx', index=False)

In [978]:
# esm_df[(esm_df['technology'] == 'solar' )] & (esm_df['country'].isin(['SE','DE']))]

esm_df_filtered = esm_df[(esm_df['technology'] == 'solar' )][['country', 'gen_perc_mix(%)', 'value_factor']]
esm_df_filtered

,country,gen_perc_mix(%),value_factor
35,SE,5.2,0.866061
26,NO,5.0,0.913233
17,FR,5.0,0.907394
8,DE,5.0,0.875276


In [979]:
gen_grouped.round(1)

,country,generation
0,DE,343075797.8
1,FR,457356011.2
2,NO,389765926.8
3,SE,86737567.0
